This workflow follows the framework of two tutorials on lidar [pre-processing](https://rapidlasso.com/2013/10/13/tutorial-lidar-preparation/) and [information extraction](https://rapidlasso.com/2013/10/20/tutorial-derivative-production/) published by Martin Isenburg. It assumes that your lidar data is in tiles and has ground returns already classified. 

In [1]:
import os, shutil, glob, platform, subprocess
import geopandas as gpd, pandas as pd
import rasterio
from matplotlib import pyplot as plt
from pyFIRS.utils import lastools, fusion, formatters
import ipyparallel as ipp
from tqdm import tqdm_notebook
import time

In [2]:
# for LAStools command line tools
las_src = '/storage/lidar/LAStools/bin/' # wherever they live
las = lastools.useLAStools(las_src)
# for FUSION command line tools
fus_src = '/storage/lidar/FUSION/' # wherever they live
fus = fusion.useFUSION(fus_src)

## Specify some key parameters for the processing pipeline

In [3]:
# where the raw lidar data is currently stored
raw_tiles = '/storage/lidar/Swinomish_Lidar_2016/source/*.laz'
workdir = os.path.abspath('/storage/lidar/Swinomish_Lidar_2016')

num_cores=32 # how many cores to use for parallel processing

Take a look at the format of the lidar data provided by the vendor.

In [4]:
vendor_tiles = glob.glob(raw_tiles)
las.lasinfo(i=vendor_tiles[0], 
            echo=True);


lasinfo (180812) report for '/storage/lidar/Swinomish_Lidar_2016/source/q48122D5415.laz'
reporting all LAS header entries:
  file signature:             'LASF'
  file source ID:             0
  global_encoding:            1
  project ID GUID data 1-4:   00000000-0000-0000-6557-747300004157
  version major.minor:        1.2
  system identifier:          'Quantum Spatial'
  generating software:        'LasMonkey 2.2.6'
  file creation day/year:     88/2017
  header size:                227
  offset to point data:       2392
  number var. length records: 3
  point data format:          1
  point data record length:   33
  number of point records:    36190794
  number of points by return: 17005960 12142658 5370083 1418786 228637
  scale factor x y z:         0.01 0.01 0.01
  offset x y z:               0 0 0
  min x y z:                  1151361.60 1124700.02 -253.61
  max x y z:                  1154513.99 1129337.94 380.54
variable length header record 1 of 3:
  reserved             0
 

## Set up the workspace 

In [5]:
# define data handling directories
raw, interim, processed = os.path.join(workdir,'raw'), os.path.join(workdir,'interim'), os.path.join(workdir,'processed')

## 1. Get the raw data into our working directory
First, move the tiles over to our working directory.

In [6]:
%%time
las.las2las(i=raw_tiles,
            odir=raw,
            drop_withheld=True, # drop any points flagged as withheld by vendor
            drop_class=(7,18), # drop any points classified as noise by vendor
            olaz=True,
            cores=num_cores)
print('Done moving tiles into working directory.')

Done moving tiles into working directory.
CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 2min 7s


Next, create spatial indexes for the input files to allow fast spatial queries (which we'll use for adding buffers).

In [7]:
%%time
infiles = os.path.join(raw,'*.laz')

las.lasindex(i=infiles, 
             cores=num_cores)

print("Done adding spatial indexes.")

Done adding spatial indexes.
CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 1min 7s


Then, we'll retile the data to add buffers for avoiding edge effects during processing.

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

If your data are in meters, you should change these parameters, or consider reprojecting the data to a projection that is in feet when you copy the source data into our working directory using `las2las` command at the top of this notebook.

In [7]:
%%time
infiles = os.path.join(raw, '*.laz')
odir = os.path.join(interim, 'retiled_v2')

las.lastile(i=infiles,
            tile_size=1000, # in units of lidar data
            buffer=100, # assumes units are in feet... if using meters, change to 25
            flag_as_withheld=True,
            olaz=True,
            odir=odir,
            cores=num_cores)

new_tiles = glob.glob(os.path.join(odir,'*.laz'))
print('Done retiling and adding buffers. Created {} tiles.'.format(len(new_tiles)))

Done retiling and adding buffers. Created 679 tiles.
CPU times: user 168 ms, sys: 52 ms, total: 220 ms
Wall time: 6min 6s


In [ ]:
# in case you need to save on storage space
# clean out the raw directory now that you have retiled data to work with
# shutil.rmtree(raw)

## 2. Classify points in the lidar point cloud
Remove noise and identify high vegetation and buildings.

In [8]:
%%time
infiles=os.path.join(interim, 'retiled_v2', '*.laz')
odir = os.path.join(interim, 'denoised_v2')

las.lasnoise(i=infiles, 
             remove_noise=True,
             odir=odir, 
             olaz=True, 
             cores=num_cores) # use parallel processing

print('Done denoising tiles.')

Done denoising tiles.
CPU times: user 116 ms, sys: 24 ms, total: 140 ms
Wall time: 4min 9s


Next, calculate the height aboveground for each point for use in classifying them.

In [9]:
%%time
infiles=os.path.join(interim, 'denoised_v2', '*.laz')
odir = os.path.join(interim, 'lasheight_v2')

las.lasheight(i=infiles,
              odir=odir, 
              olaz=True, 
              cores=num_cores) # use parallel processing

print('Done calculating height above ground.')

Done calculating height above ground.
CPU times: user 236 ms, sys: 60 ms, total: 296 ms
Wall time: 7min 5s


Now, we'll classify points (that haven't already been classified into meaningful categories) as building or high vegetation that meet certain criteria for 'planarity' or 'ruggedness'. 

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

If your data are in meters, you should change these parameters, or consider reprojecting the data to a projection that is in feet when you copy the source data into our working directory using `las2las` command at the top of this notebook.

In [10]:
%%time
infiles = os.path.join(interim, 'lasheight_v2', '*.laz')
odir = os.path.join(interim, 'classified_v2')

las.lasclassify(i=infiles,
                odir=odir,
                olaz=True,
                step=5, # if your data are in meters, the LAStools default is 2.0
                planar=0.5, # if your data are in meters, the LAStools default is 0.1
                rugged=1, # if your data are in meters, the LAStools default is 0.4
                ignore_class=(2,9,10,11,13,14,15,16,17), # ignore points already classified meaningfully
                cores=num_cores) # use parallel processing

print('Done classifying lidar tiles.')

Done classifying lidar tiles.
CPU times: user 224 ms, sys: 72 ms, total: 296 ms
Wall time: 28min 28s


Finally, remove buffer points from the classified tiles and put the clean tiles in the processed folder.

In [12]:
%%time
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'points')

las.las2las(i=infiles,
            odir=odir,
            olaz=True,
            drop_withheld=True, # remove points in tile buffers that were flagged as withhled with lastile
            set_user_data=0, # remove height aboveground calculated using lasheight
            cores=num_cores)

print('Done trimming classified lidar tiles.')

Done trimming classified lidar tiles.
CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 2min 16s


Produce a shapefile showing the layout of the tiles of points.

In [13]:
%%time
infiles = os.path.join(processed, 'points', '*.laz')
odir = os.path.join(processed, 'vectors')

las.lasboundary(i=infiles,
                odir=odir,
                o='tiles.shp',
                oshp=True,
                use_bb=True, # use bounding box of tiles
                overview=True,
                labels=True,
                cores=num_cores) # use parallel processing

print('Produced a shapefile overview of clean tile boundaries.')

Produced a shapefile overview of clean tile boundaries.
CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 4.8 s


In [24]:
# %%time
# remove intermediate lidar files if you want to reclaim storage space
# shutil.rmtree(os.path.join(interim, 'retiled'))
# shutil.rmtree(os.path.join(interim, 'denoised'))
# shutil.rmtree(os.path.join(interim, 'lasheight'))

## 3. Generate a bare earth Digital Elevation Model
Generate tiles of the bare earth model. This assumes that there are already ground-classified points

In [208]:
%%time
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'rasters', 'DEM_tiles')

las.las2dem(i=infiles,
            odir=odir,
            otif=True, # create tiles as GeoTiff rasters
            keep_class=2, # keep ground-classified returns only
            thin_with_grid=1, # use a 1 x 1 resolution for creating the TIN for the DEM
            step=1, # grid cell size to use for raster, in units of lidar data
            extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
            use_tile_bb=True, # remove buffers from tiles
            cores=num_cores) 

# specify the projection for all GeoTiffs using rasterio command line tool
for file in glob.glob(os.path.join(processed, 'rasters', 'DEM_tiles', '*.tif')):
    subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:2286', file],
                   stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    
print('Done producing bare earth tiles.')

Done producing bare earth tiles.
CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 2min 26s


In [225]:
dir_to_clean = os.path.join(processed,'rasters','DEM_tiles')
clean_dir(dir_to_clean, ['.kml', '.tfw'])

Removed 76 files with extension .kml.
Removed 76 files with extension .tfw.


Merge the bare earth tiles into a single DEM formatted as a GeoTiff.

In [209]:
%%time
infiles = os.path.join(processed, 'rasters', 'DEM_tiles', '*.tif')
outfile = os.path.join(processed, 'rasters', 'dem.tif')

# merge the GeoTiffs using rasterio command line tool
subprocess.run(['rio', 'merge', *glob.glob(infiles), outfile, '--co', 'compress=LZW'],
               stderr=subprocess.PIPE, stdout=subprocess.PIPE)

print('Done producing merged DEM GeoTiff.')

Done producing merged DEM GeoTiff.
CPU times: user 8 ms, sys: 692 ms, total: 700 ms
Wall time: 1min 22s


To create a hillshade layer, we'll first, generate hillshade tiles from the bare earth model.

In [210]:
%%time
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'rasters', 'hillshade_tiles')

las.las2dem(i=infiles,
            odir=odir,
            otif=True, # create tiles as GeoTiffs
            cores=num_cores,
            hillshade=True,
            keep_class=2, # keep ground-classified returns only
            thin_with_grid=1, # use a 0.5 x 0.5 resolution for creating the TIN for the DEM
            extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
            step=1, # grid cell size to use for raster, in units of lidar data
            use_tile_bb=True) # remove buffers from tiles

# specify the projection for all GeoTiffs using rasterio command line tool
for file in glob.glob(os.path.join(processed, 'rasters', 'hillshade_tiles', '*.tif')):
    subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:2286', file],
                   stderr=subprocess.PIPE, stdout=subprocess.PIPE)

print('Done producing hillshade bare earth tiles.')

Done producing hillshade bare earth tiles.
CPU times: user 12 ms, sys: 16 ms, total: 28 ms
Wall time: 2min 45s


In [226]:
dir_to_clean = os.path.join(processed,'rasters','hillshade_tiles')
clean_dir(dir_to_clean, ['.kml', '.tfw'])

Removed 76 files with extension .kml.
Removed 76 files with extension .tfw.


Now merge the hillshade tiles into a single raster formatted as GeoTiff.

In [211]:
%%time
infiles = os.path.join(processed, 'rasters', 'hillshade_tiles', '*.tif')
outfile = os.path.join(processed, 'rasters', 'hillshade.tif')

# merge the GeoTiffs using rasterio command line tool
subprocess.run(['rio', 'merge', *glob.glob(infiles), outfile, '--co', 'compress=LZW'],
               stderr=subprocess.PIPE, stdout=subprocess.PIPE)

print('Done producing merged hillshade GeoTiff.')

Done producing merged hillshade GeoTiff.
CPU times: user 12 ms, sys: 696 ms, total: 708 ms
Wall time: 1min 59s


## 4. Identify building footprints
First start by building shapefiles showing building boundaries in each buffered tile.

In [18]:
%%time
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(interim, 'building_tiles')

las.lasboundary(i=infiles,
                odir=odir,
                keep_class=6, # use only building-classified points
                disjoint=True, # compute separate polygons for each building
                concavity=3, # map concave boundary if edge length >= 3ft
                cores=num_cores)

print('Done producing building footprints in buffered tiles.')

Done producing building footprints in buffered tiles.
CPU times: user 12 ms, sys: 16 ms, total: 28 ms
Wall time: 1min 15s


Generate shapefiles showing the bounding box of each (unbuffered) tile that we'll use to remove buildings that fall in the buffered area.

In [19]:
%%time
infiles = os.path.join(processed, 'points', '*.laz')
odir = os.path.join(interim, 'tile_boundaries')

las.lasboundary(i=infiles,
                odir=odir,
                oshp=True,
                use_tile_bb=True,
                cores=num_cores)

print('Done producing boundaries of unbuffered tiles.')

Done producing boundaries of unbuffered tiles.
CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 5.74 s


For each shapefile containing polygons of the building boundaries, we'll use the `clean_tile` function to remove polygons from a tile if their centroid falls in the buffered area of the tile.

In [20]:
%%time
building_tiles = glob.glob(os.path.join(interim, 'building_tiles', '*.shp'))
odir = os.path.join(processed, 'vectors', 'building_tiles')

for poly_shp in building_tiles:
    fname = os.path.basename(poly_shp)
    tile_shp = os.path.join(interim, 'tile_boundaries', fname)
    lastools.clean_buffer_polys(poly_shp, tile_shp, odir, simp_tol=3, simp_topol=True)

print('Done producing building footprints in cleaned (unbuffered) tiles.')

Done producing building footprints in cleaned (unbuffered) tiles.
CPU times: user 11.4 s, sys: 4 ms, total: 11.4 s
Wall time: 12.1 s


Merge the cleaned tiles of building footprints together into a single shapefile. We'll use geopandas to concatenate all the polygons together into a single geodataframe and then write out to a new shapefile.

In [21]:
%%time
building_tiles = glob.glob(os.path.join(processed, 'vectors', 'building_tiles', '*.shp'))
# create a list of geodataframes containing the tiles of building footprints
gdflist = [gpd.read_file(tile) for tile in building_tiles]
# merge them all together
merged = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True))
# using pandas concat caused us to lose projection information, so let's add that back in
merged.crs = gdflist[0].crs
# and write the merged data to a new shapefile
merged.to_file(os.path.join(processed,'vectors','buildings.shp'))

print('Done merging tiles of building footprints into a single shapefile.')

Done merging tiles of building footprints into a single shapefile.
CPU times: user 4.29 s, sys: 12 ms, total: 4.3 s
Wall time: 4.32 s


## 4. Create a Canopy Height Model

We'll switch to using `FUSION` command line tools to generate a Canopy Height Model (CHM). 

Because `FUSION` commands don't have built-in support for multi-core processing in each command line tool, to run processing in parallel, we'll launch a parallel computing cluster using `ipyparallel`. If you have `ipyparallel` installed in the computing environment that was used to launch this notebook (which may be different from the kernel you're using to execute it), you should be able to start a parallel computing cluster by switching to the "IPython Clusters" tab of the "Home" tab that was created when you called `jupyter notebook` from the console.  Note that `pyFIRS` needs to be installed in the virtual environment used to launch this notebook.

Alternatively, you can also issue a call from the command line to fire up a cluster of workers. You will need to have your virtual environment activated (`activate env_name` on Windows or `source activate env_name` on other OSs, replacing `env_name` with whatever your environment's name is), and then call:

```> ipcluster start -n #``` where # is the number of cores you want to use

In [198]:
rc = ipp.Client() # a client for controlling the workers
print(rc.ids) # how many workers?

dv = rc[:] # create a direct view of the workers
v = rc.load_balanced_view() # create a load-balanced view of the workers

# import the relevant packages to all the workers
with dv.sync_imports():
    import subprocess
    import os
    from pyFIRS.utils import lastools
    from pyFIRS.utils import fusion
    import rasterio
    import geopandas
    
# push some objects to the workers
dv.push(dict(raw=raw, interim=interim, processed=processed, las_src=las_src, fus_src=fus_src))


# instantiate the useLAStools class on the workers 
# so they can each execute LAStools commands
%px las = lastools.useLAStools(las_src) 
%px fus = fusion.useFUSION(fus_src)

/storage/anaconda3/envs/pyFIRS/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@Ford')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
importing subprocess on engine(s)
importing os on engine(s)
importing lastools from pyFIRS.utils on engine(s)
importing fusion from pyFIRS.utils on engine(s)
importing rasterio on engine(s)
importing geopandas on engine(s)


In [11]:
# push a unique identifier to each worker that we'll use to run separate instances of WINE
prefixes = [x for x in range(num_cores)]
dv.scatter('wine_prefix', prefixes)
print(dv['wine_prefix'])

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31]]


In [12]:
# create the wineprefixes in case they doesn't already exist
%px subprocess.run(['export WINEPREFIX=~/.wine-{}'.format(wine_prefix[0])], shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE);

If you want to track progress while an asynchronous batch is running, you can execute the following function to show a progress bar including estimated time to completion. This will lock the notebook (i.e., no other notebook cells may be executed) until the batch is completed unless you interrupt the kernel.

In [194]:
def pbar(res):
    jobs_done = res.progress
    with tqdm_notebook(total=len(res), initial=jobs_done, desc='Progress', unit='tile') as pbar:
        while not res.ready():
            new_progress = res.progress - jobs_done
            jobs_done += new_progress
            pbar.update(new_progress)
            time.sleep(0.5)
        # once jobs are completed (i.e., res.ready() returns True)
        # update the progress bar one last time
        pbar.update(len(res)-jobs_done)

We will be mapping a single function to a list of inputs, and only want to specify the keyword arguments once. We'll create a wrapper function which specifies the keyword arguments and takes a single input (a tile of classified lidar data) to produce a CHM.

On Linux machines, we need to utilize separate instances of WINE for parallel processing using the Windows executables. By specifying `wine_prefix`, separate WINE servers will be spun up by each worker to execute the job sent to it. 

### Using FUSION's `canopymodel`
We'll first normalize all the tiles to height above ground so we don't need to provide a ground DTM for FUSION.

In [10]:
%%time
infiles=os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(interim, 'normalized')

las.lasheight(i=infiles,
              odir=odir, 
              olaz=True, 
              replace_z=True,
              drop_below=0.1,
              cores=num_cores) # use parallel processing

print('Done normalizing tiles with ground and vegetation.')

Done normalizing tiles with ground and vegetation.
CPU times: user 96 ms, sys: 40 ms, total: 136 ms
Wall time: 6min 47s


Now we'll create a function that we'll map to a list of input files and distribute to our workers in parallel.

In [49]:
def run_canopymodel(infile):
    odir = os.path.join(interim, 'chm_tiles')
    outname = os.path.basename(infile).split('.')[0] + '.dtm'
    outfile = os.path.join(odir, outname)
    return fus.canopymodel(surfacefile=outfile,
                           odir=odir,
                           cellsize=1,
                           xyunits='F',
                           zunits='F',
                           coordsys=2, # in State Plane
                           zone=0, # not in UTM
                           horizdatum=2, # NAD83
                           vertdatum=2, # NAVD88
                           datafiles=infile,
                           median=3, # median smoothing in nxn kernel
                           las_class=(1,2,5), # keep only ground, unclassified, and high veg points
                           asc=True) # also output in ascii format

In [50]:
tiles_to_run = glob.glob(os.path.join(interim, 'normalized', '*.laz'))

# run the batch
res = v.map_async(run_canopymodel, tiles_to_run)
print('Started batch run, processing {} tiles.'.format(len(tiles_to_run)))
pbar(res)

Started batch run, processing 76 tiles.


In [224]:
def clean_dir(dir_to_clean, file_extensions):
    '''Deletes files with specified extension(s) from a directory.
    
    This function is intended to help cleanup outputs from command line
    tools that we do not want to keep. Files to be deleted will be 
    identified using a wildcard with that file extension in dir_to_clean. 
    
    Parameters
    ----------
    dir_to_clean: string, path
        path to directory to delete files from
    file_extension: string or list-like of strings
        file extensions that will be used for identifying files to remove,
        such as ['.tfw', '.kml'].
    '''
    for ext in file_extensions:
        to_rem = glob.glob(os.path.join(dir_to_clean, '*{}'.format(ext)))
        for file in to_rem:
            os.remove(file)
        print("Removed {:,d} files with extension {}.".format(len(to_rem),ext))

In [175]:
def convert_project(infile):
    '''Converts a raster to GeoTiff and specifies a Coordinate Reference System.'''
    indir, file = os.path.split(infile)
    fname = file.split('.')[0]
    outfile = os.path.join(indir, fname + '.tif')
    proc_trans = subprocess.run(['rio', 'convert', infile, outfile], 
                                stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    proc_project = subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:2286', outfile],
                                 stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    return proc_trans, proc_project

In [178]:
tiles_to_run = glob.glob(os.path.join(interim, 'chm_tiles_FUSION', '*.asc'))

# run the batch
res = v.map_async(convert_project, tiles_to_run, crs='EPSG:2286')
print('Started batch run, processing {} tiles.'.format(len(tiles_to_run)))

Started batch run, processing 76 tiles.


In [179]:
jobs_done = res.progress
with tqdm_notebook(total=len(res), initial=jobs_done, desc='Progress', unit='tile') as pbar:
    while not res.ready():
        new_progress = res.progress - jobs_done
        jobs_done += new_progress
        pbar.update(new_progress)
        time.sleep(0.5)
    # once jobs are completed (i.e., res.ready() returns True)
    # update the progress bar one last time
    pbar.update(len(res)-jobs_done)

In [55]:
%%time 
# delete the dtm files
dtm_files = os.path.join(interim, 'chm_tiles_FUSION', '*.dtm')
del_files = glob.glob(dtm_files)
for file in del_files:
    os.remove(file)

CPU times: user 0 ns, sys: 476 ms, total: 476 ms
Wall time: 474 ms


In [156]:
jobs_done = res.progress
with tqdm_notebook(total=len(res), initial=jobs_done, desc='Progress', unit='tile') as pbar:
    while not res.ready():
        new_progress = res.progress - jobs_done
        jobs_done += new_progress
        pbar.update(new_progress)
        time.sleep(0.5)
    # once jobs are completed (i.e., res.ready() returns True)
    # update the progress bar one last time
    pbar.update(len(res)-jobs_done)

In [202]:
def clip_tile(infile):
    fname = os.path.basename(infile).split('.')[0]
    tile_shp = os.path.join(interim, 'tile_boundaries', fname+'.shp')
    gdf = geopandas.read_file(tile_shp)
    tile_bnds = ' '.join(str(x) for x in gdf.total_bounds)
    odir = os.path.join(processed, 'rasters', 'chm_tiles')
    os.makedirs(odir, exist_ok=True)
    outfile = os.path.join(odir, fname+'.tif')
    
    proc_clip = subprocess.run(['rio', 'clip', infile, outfile, '--bounds', tile_bnds],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    return proc_clip

In [203]:
tiles_to_run = glob.glob(os.path.join(interim, 'chm_tiles_FUSION', '*.tif'))

# run the batch
res = v.map_async(clip_tile, tiles_to_run)
print('Started batch run, processing {} tiles.'.format(len(tiles_to_run)))
pbar(res)

Started batch run, processing 76 tiles.


In [207]:
%%time
infiles = glob.glob(os.path.join(processed, 'rasters', 'chm_tiles', '*.tif'))
outfile = os.path.join(processed, 'rasters', 'chm.tif')
subprocess.run(['rio', 'merge', *infiles, outfile, '--co', 'compress=LZW'],
               stderr=subprocess.PIPE, stdout=subprocess.PIPE)

CPU times: user 4 ms, sys: 12 ms, total: 16 ms
Wall time: 1min 20s
